## Testing connection between Jupyter and the Spark Cluster

The main goal of this notebook is to test if the connection between the Jupyter service and the Spark Cluster is working properly.
This could be used as a snippet for any other Jupyter-Spark development.

### Importing libraries

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

### Connecting to Spark cluster and raising a Spark Session

Once the session is raised, we should be able to see the app name in http://localhost:8080/.

In [3]:
master = "spark://spark:7077"
app_name = "Testing if jupyter can communicate with spark"
spark = (
    SparkSession.builder
    .appName(app_name)
    .master(master)
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)
sc = spark.sparkContext
sc.setLogLevel("WARN")

print("Spark version: " + str(sc.version))

Spark version: 3.2.0


### Terminating Spark session

Otherwise, it will be endlessly running.

In [ ]:
spark.stop()